# Flow Metrics

This workbook generates reports on flow metric from a given GitHub repo using the Github API.

Author: [Brian McIlwain](mailto:brian@poq.gg)

## Setup

This gets data from the [Github GraphQL API](https://docs.github.com/en/graphql/overview/about-the-graphql-api) to create reports for flow metrics

### Imports, utils, & globals

In [1]:
from dateutil.relativedelta import relativedelta, SU
from datetime import date
from functools import reduce
import numpy as np
import pandas as pd
from gql.transport.aiohttp import AIOHTTPTransport
from gql import gql, Client
from os import environ
from dotenv import load_dotenv

load_dotenv()
API_SECRET_TOKEN = environ.get('API_SECRET_TOKEN')

if not API_SECRET_TOKEN:
    raise Exception(
        'API_SECRET_TOKEN is not defined as an environmental variable')

# Config for repo
API_URL = "https://api.github.com/graphql"
REPO_OWNER = "weiks"
REPO_NAME = "esports-backend"
MAX_WEEK_ISSUES = 100  # API limit, shouldn't be hit
MAX_PAGINATION_LIMIT = 100  # Set by API, I wish it was larger
MAX_LABEL_LIMIT = 20  # Max labels per PR

print(f"Repo: {REPO_OWNER}/{REPO_NAME}\n")

# Prep to run GQL calls
headers = {"Authorization": f"Bearer {API_SECRET_TOKEN}"}
transport = AIOHTTPTransport(url=API_URL, headers=headers)


async def runQuery(query, variable_values=None):
    # Create a GraphQL client using the defined transport
    # Using `async with` on the client will start a connection on the transport
    # and provide a `session` variable to execute queries on this connection
    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:
        # Execute the query on the transport
        result = await session.execute(query, variable_values)
        return result


def get_previous_sunday(working_date=date.today()):
    last_sunday = working_date + relativedelta(weekday=SU(-1))
    return last_sunday.strftime("%Y-%m-%d")


def unwrap_name(d):
    return d['name']


def unwrap_login(d):
    return d['login']


def clean_issue_data(issue):
    issue['labels'] = set(map(unwrap_name, issue['labels']['nodes']))
    issue['assignees'] = set(map(unwrap_login, issue['assignees']['nodes']))
    return issue


def clean_PR_data(pr):
    pr['author'] = pr['author']['login']
    pr['assignees'] = set(map(unwrap_login, pr['assignees']['nodes']))
    pr['assignees'].add(pr['author'])  # Author is an assignee by default
    pr['issues'] = list(map(clean_issue_data, pr['issues']['nodes']))
    pr['labels'] = set(map(unwrap_name, pr['labels']['nodes']))
    pr['labels'].discard('trigger-ci')  # Ignore trigger-ci

    # Apply parent issue labels to the PR
    if pr['issues']:
        issues_labels = list(map(lambda issue: issue['labels'], pr['issues']))
        flat_issues_labels = reduce(lambda a, b: a.union(b), issues_labels)
        pr['labels'] = pr['labels'].union(flat_issues_labels)

    return pr


Repo: weiks/esports-backend



## Get PRs Data

In [ ]:
async def get_PR_data():
    cursor = ""
    prs = []
    print('Fetching PRs, this can take a while...')

    while cursor != None:
        overviewQuery = gql(
            f"""
            query getRepoData {{
              repository(owner: "{REPO_OWNER}", name: "{REPO_NAME}") {{
                pullRequests(
                  first: {MAX_PAGINATION_LIMIT},
                  orderBy: {{ field: UPDATED_AT, direction: DESC }}
                  { f'after: "{cursor}"' if cursor else '' }
                ) {{
                  pageInfo {{
                    hasNextPage
                    endCursor
                  }}
                  nodes {{
                    title
                    author {{
                      login
                    }}
                    assignees(first: 10) {{
                      nodes {{
                        login
                      }}
                    }}
                    number
                    url
                    state
                    closedAt
                    createdAt
                    updatedAt
                    labels(first: {MAX_LABEL_LIMIT}) {{
                      nodes {{
                        name
                      }}
                    }}
                    issues: closingIssuesReferences(first: {MAX_LABEL_LIMIT}) {{
                      nodes {{
                        assignees(first: 10) {{
                          nodes {{
                            login
                          }}
                        }}
                        labels(first: {MAX_LABEL_LIMIT}) {{
                          nodes {{
                            name
                          }}
                        }}
                        number
                        title
                        url
                        state
                        updatedAt
                      }}
                    }}
                  }}
                }}
              }}
            }}
            """)
        result = await runQuery(overviewQuery)
        prs += result['repository']['pullRequests']['nodes']
        cursor = result['repository']['pullRequests']['pageInfo']['endCursor']
    prs = list(map(clean_PR_data, prs))
    prsDF = pd.DataFrame(prs)
    prsDF.to_csv('prs_data.csv', index=False)
    return prsDF

await get_PR_data()


## Get Issue Data

In [ ]:
async def get_issue_data():
    cursor = ""
    issues = []
    print('Fetching issues, this can take a while...')

    while cursor != None:
        overviewQuery = gql(
            f"""
            query getRepoData {{
              repository(owner: "{REPO_OWNER}", name: "{REPO_NAME}") {{
                issues(
                  first: {MAX_PAGINATION_LIMIT}, 
                  orderBy: {{ field: UPDATED_AT, direction: DESC }}
                  { f'after: "{cursor}"' if cursor else '' }
                ) {{
                  pageInfo {{
                    hasNextPage
                    endCursor
                  }}
                  nodes {{
                    title
                    author {{
                      login
                    }}
                    assignees(first: 10) {{
                      nodes {{
                        login
                      }}
                    }}
                    number
                    url
                    state
                    closedAt
                    createdAt
                    updatedAt
                    labels(first: {MAX_LABEL_LIMIT}) {{
                      nodes {{
                        name
                      }}
                    }}
                  }}
                }}
              }}
            }}
            """)
        result = await runQuery(overviewQuery)
        issues += result['repository']['issues']['nodes']
        cursor = result['repository']['issues']['pageInfo']['endCursor']
    issues = list(map(clean_issue_data, issues))
    issuesDF = pd.DataFrame(issues)
    issuesDF.to_csv('issues_data.csv', index=False)
    return issuesDF

await get_issue_data()
